# EM4QT: EM for Query Translation in CLIR

In this paper, the authors propose a new method to refine the query language model based on top-ranked documents. In CLIR setting, we can have both source language documents and target language documents. Source language documents are retrieved in response to the native speaker subject, and the target language documents are retrieved in response to a translation of it. 

In example below let's define $q$ is the inital query posed by the native subject and $qt$ is a simple translation using a bilingual dictionary. $d$ is defined to be the first 8 documents retrieved after $q$ and $d_t$ is analogous documents for $q_t$.

In [5]:
import numpy as np
import nltk
from nltk.corpus import reuters
from nltk.corpus import brown
from collections import Counter
from nltk import word_tokenize 
import urllib2
import unidecode
from urllib2 import urlopen



nltk.download('reuters')
nltk.download('brown')
nltk.download('punkt')
#raw = urllib2.urlopen("http://www.gutenberg.org/cache/epub/13704/pg13704.txt")
response = urlopen("http://www.gutenberg.org/cache/epub/13704/pg13704.txt")
data = response.read()

ds=['A 360° photo was taken at both the Opening and Closing matches at Russia 2018. Were you there? Scroll, zoom, tag and share! Amazing Pictures',
   '2018 FIFA World Cup Russia - Players. ... Uruguay. FRA. Didier DESCHAMPS. France. BRA. TITE. Brazil. RUS. Stanislav CHERCHESOV. Russia',
   'The 2018 FIFA World Cup was the 21st FIFA World Cup, an international football tournament ... As such, there were eventually four bids for the 2018 FIFA World Cup, two of which were joint bids: England, Russia, Netherlands/Belgium, and ...',
   'Russia has participated in 4 FIFA World Cups since its independence in December 1991. The Russian Federation played their first international match against ..',
   'Russia s bad press is largely of its own making – for years, it has been easier for officials to bray about Russophobia than to show a different',
   'The 2018 World Cup has been seen as a public relations triumph for Vladimir Putin\'s Russia, but beyond the football concerns remain about',
   'It has been a wonderful ride, but the remarkable World Cup run that set millions of people dreaming across Russia\'s 11 time zones is finally',
   'Information about FIFA World Cup Russia 2018. Select the subjects you want to know more about within all the international subjects treated on euronews.com']

dt=['Du 14 juin au 15 juillet 2018, la Russie organise la 21e edition de la Coupe du monde de football. Trente-deux equipes s\'affrontent pour savoir',
   ' Du 14 juin au 15 juillet, 32 equipes s\'affronteront pour conquerir le titre de champion du monde de football. Calendrier, « bilan de competences ...',
   ' Lors du tirage au sort de la Coupe du monde 2018, debut decembre, le pays ... By universal acclaim, the FIFA Russia 2018 world cup poster ... du XXe siecle, et le pouvoir a mis un demi-siecle a mettre le grappin dessus.',
   'Saransk est l\'une des villes hotes de la Coupe du Monde de la FIFA, Russie 2018 ... de Rubrouck, recit de voyage a travers les terres de l\'Est, XIIIeme siecle) *.',
   'Les hommes portent des habits dont la coupe remonte jusqu\'au temps de Ruyter, ... et qui ne leur laissait avec le monde civilise d\'autre relation que la guerre. ... ou envoyes de tous les points du globe par des admirateurs du grand homme. ... Il faut le dire, Pierre le Grand partage l\'admiration de l\'univers avec Louis XIV,',
   'Preparez votre sejour dans les Bouches-du Rhone. My Provence a selectionne pour vous le meilleur de l\'offre touristique du departement.',
   'Revenant sur les critiques sur ses chutes a repetition pendant la Coupe du monde, Neymar admet des « exagerations » dans une publicite ou il se presente .',
   'Coupe du monde 2018 de football - suivez en direct les matchs, scores et resultats du Mondial ainsi que toutes les infos (compositions, tableau, calendrier…']
#lines = txt_str.split("\\n")

[nltk_data] Downloading package reuters to /home/javid/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package brown to /home/javid/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/javid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Preprocessing Data
Here I'm going to spend a little bit of time to unify the accent characters in the french data and tokenize it to build a collection languag eodel 

In [6]:
import unicodedata

ds = [x.lower() for x in ds]
dt = [x.lower() for x in dt]

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
french_corpus = strip_accents(data)

# Building Collection Languae Model

Here we are using a large corpus to compute language models in English and French. Since this is a demo and I don't want to crawl a really large corpus, it's possible to miss some words in collection. So I concatenate the collection and the feedback documents together for the collection language model to prevent missing some words in final analysis.

In [7]:
from nltk import SimpleGoodTuringProbDist, FreqDist

p_c = Counter(reuters.words()+brown.words()) + Counter(word_tokenize(' '.join(ds)))
total_count_c = len(reuters.words()+brown.words()+word_tokenize(' '.join(ds)))

p_c_t = Counter(word_tokenize(french_corpus+' '.join(dt)))
total_count_c_t = len(word_tokenize(french_corpus+' '.join(dt)))


# Compute the frequencies
for word in p_c:
    p_c[word] /= float(total_count_c)

for word in p_c_t:
    p_c_t[word] /= float(total_count_c_t) 

# Building Feedback Language Model

Here first we are doing a retrieval run to obtain collections $d_t$ and $d_s$. Then we estimate the feedback model using these collections. 

In [8]:
p_t_ws_wt={}
p_wt_t_ws={}

lambda_ = 0.9
q = ['world','cup','russia']
qt=[('monde',0.25),('univers',0.25),('globe',0.25),('siecle',0.25),('tasse',0.5),('coupe',0.5),('russia',1.0)]

p_wt_t_ws[('world','monde')]=0.25
p_wt_t_ws[('world','univers')]=0.25
p_wt_t_ws[('world','globe')]=0.25
p_wt_t_ws[('world','siecle')]=0.25
p_wt_t_ws[('cup','tasse')]=0.5
p_wt_t_ws[('cup','coupe')]=0.5
p_wt_t_ws[('russia','russia')]=1.0

p_t_ws_wt[('world','monde')]=0.5
p_t_ws_wt[('world','univers')]=0.5
p_t_ws_wt[('world','globe')]=0.5
p_t_ws_wt[('world','siecle')]=0.5
p_t_ws_wt[('cup','tasse')]=0.5
p_t_ws_wt[('cup','coupe')]=0.5
p_t_ws_wt[('russia','russia')]=0.5

p_f = Counter(word_tokenize(' '.join(ds)))
p_f_t = Counter(word_tokenize(' '.join(dt)))

#p_f = Counter(word_tokenize(d))
total_count_f = len(word_tokenize(' '.join(ds)))
total_count_f_t = len(word_tokenize(' '.join(dt)))

# Compute the frequencies
for word in p_f:
    p_f[word] /= float(total_count_f)
for word in p_f_t:
    p_f_t[word] /= float(total_count_f_t)

# EM algorithm

Finally we estimate the translation probabilities using the following expectation maximization algorithm:


$$p(T=1|w_s) = \frac{p(w_s|T=1) p(T=1)}{p(w_s)}$$

$$p(w_s) = p(w_s|T=1)\lambda + p(w_s|T=0)(1-\lambda)$$

$$=p(w_s|\theta_F)\lambda + \frac{p(T=0|w_s)p(w_s)(1-\lambda)}{p(T=0)} $$
 $$= p(w_s|\theta_F)\lambda + p(T=0|w_s)p(w_s)$$

$$p(T=0|w_s) = (1-\frac{p(w_s|\theta_F)}{p(w_s|\mathcal{C})}\lambda)$$



$$p^{(i)}(T=1|w_s, w_t) = \frac{\lambda p^{(i)}(w_t|T=1, w_s)}{(1-\frac{p(w_s|\theta_F)}{p(w_s|\mathcal{C}_s)}\lambda) p(w_t|\mathcal{C}_t) + \lambda p^{(i)}(w_t|T=1, w_s)}$$

$$p^{(i+1)}(w_t|T=1, w_s) = \frac{p(w_t|\theta_{F'}) p^{(i)}(T=1|w_s, w_t)}{\sum_{w'_t \in \mathscr{T}\{w_s\}}{p(w'_t|\theta_{F'}) p^{(i)}(T=1|w_s, w'_t)}}$$

In [9]:
for it in range(20):
    from collections import defaultdict
    p_ws = {}
    p_ws = defaultdict(lambda:0,p_ws)
    for ws,wt in p_t_ws_wt:
        if(p_wt_t_ws[(ws,wt)]):
            p_t_ws_wt[(ws,wt)] = lambda_*p_wt_t_ws[(ws,wt)]/((1-p_f[ws]/p_c[ws])*p_c_t[wt]+lambda_*p_wt_t_ws[(ws,wt)])
    for ws,wt in p_wt_t_ws:
        p_ws[ws] +=  p_f_t[wt]*p_t_ws_wt[(ws,wt)]
    for ws,wt in p_wt_t_ws:
        if p_ws[ws]:
            p_wt_t_ws[(ws,wt)] = p_f_t[wt]*p_t_ws_wt[(ws,wt)]/(p_ws[ws]);
        
print(p_wt_t_ws)


{('world', 'globe'): 0.09486017699528382, ('world', 'siecle'): 0.1981119017533696, ('cup', 'coupe'): 1.0, ('world', 'monde'): 0.7070279212513466, ('world', 'univers'): 0.0, ('cup', 'tasse'): 0.0, ('russia', 'russia'): 1.0}


[1] Javid Dadashkarimi, Azadeh Shakery, Heshaam Faili, and Hamed Zamani. An Expectation-Maximization Algorithm for Query Translation Based on Pseudo-Relevant Documents. Journal of Information Processing & Management, Elsevier, 2017 Journal Paper.